# B+ Trees

In [1]:
class BPTree(object):
    def __init__(self, internal_capacity, leaf_capacity):
        self.internal_capacity = internal_capacity
        self.leaf_capacity = leaf_capacity
        self.root = InternalNode(internal_capacity, leaf_capacity)
    
    def __repr__(self):
        return "Root: " + str(self.root)
    
    def split_root(self):
        """
        Creates a new root node and places the split current rood node
        as its children
        """
        promote, new_right = self.root.split()
        
        new_root = InternalNode(self.internal_capacity, self.leaf_capacity)
        new_root.items.append(promote)
        new_root.children.append(self.root)
        new_root.children.append(new_right)
        
        self.root = new_root
    
    def insert(self, tup):
        """
        Inserts tuple into B+Tree, handles splitting if root node
        is over capacity
        """
        self.root.insert(tup)
        if self.root.over_capacity():
            self.split_root()
            
    def in_order_traversal(self):
        """ Returns the tuples from the smallest to largest """
        node = self.root
        while type(node) is not LeafNode:
            node = node.children[0]
        while node is not None:
            for tup in node.items:
                yield tup
            node = node.rsibling
            
    def depth(self):
        """ Computes the depth of the B+Tree """
        d = 0
        node = self.root
        while type(node) is not LeafNode:
            node = node.children[0]
            d += 1
        return d

In [2]:
class InternalNode(object):
    def __init__(self, internal_capacity, leaf_capacity):
        self.capacity = internal_capacity
        self.leaf_capacity = leaf_capacity
        self.items = []
        self.children = []
        
    def __repr__(self):
        s = ''
        s += 'Internal Node\n'
        s += f'Capacity: {self.capacity}\n'
        s += f'Items: {self.items}\n'
        s += f'Children: ['
        for child in self.children:
            c = ''.join(['    ' + x + '\n' for x in str(child).splitlines()])
            s += f'\n{c}'
        s = s.rstrip(',')
        s += ']'
        return s
    
    def over_capacity(self):
        """ Returns True if node is over capacity """
        return len(self.items) > self.capacity
    
    def insert(self, tup):
        """
        insert handles insertion into both internal and leaf nodes by 
        calling the appropriate functions
        """
        if len(self.children) == 0:
            self.children.append(LeafNode(self.leaf_capacity))
        if isinstance(self.children[0], LeafNode):
            self.insert_into_leaf(tup)
        elif isinstance(self.children[0], InternalNode):
            self.insert_into_internal(tup)
        else:
            raise Exception(f"Children of type: {type(self.children[0])}")
            
    def split(self):
        """
        copes half of this internal node's items and children into a new internal
        node - new_right
        """
        new_right = InternalNode(self.capacity, self.leaf_capacity)
        split_idx = self.capacity//2
        promote = self.items[split_idx]
        new_right.items = self.items[split_idx+1:]
        new_right.children = self.children[split_idx+1:]
        self.items = self.items[:split_idx]
        self.children = self.children[:split_idx+1]

        return promote, new_right
    
    def insert_into_internal(self, tup):
        """
        insert_into_internal inserts a tuple into a child internal node
        it will also split that child node if it is over capacity
        """
        # index of child internal node
        child_idx = len(self.items)
        for i, item in enumerate(self.items):
            if tup < item:
                child_idx = i
                break
        node = self.children[child_idx]
        
        # insert into node
        node.insert(tup)
        
        if node.over_capacity():
            promote, new_right = node.split()
            self.items.insert(child_idx, promote)
            self.children.insert(child_idx + 1, new_right)
            
    def split_leaf(self, leaf, child_idx):
        """
        splits a leaf into two nodes. Also inserts new split node as a child of
        this current internal node
        """
        new_right = leaf.split()
        self.items.insert(child_idx, new_right.items[0])
        self.children.insert(child_idx + 1, new_right)
            
    def insert_into_leaf(self, tup):
        """ inserts a tuple into appropriate child leaf node """
        # index of child leaf node
        child_idx = len(self.items)
        for i, item in enumerate(self.items):
            if tup < item:
                child_idx = i
                break
        leaf = self.children[child_idx]
        
        # insert tuple into leaf node
        leaf.insert(tup)
        
        # fix leaf node if it is over capacity
        if leaf.over_capacity():
            self.split_leaf(leaf, child_idx)

In [3]:
class LeafNode(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.items = []
        self.rsibling = None
        
    def __repr__(self):
        s = ''
        s += 'Leaf Node\n'
        s += f'Capacity: {self.capacity}\n'
        s += f'Items: {self.items}\n'
        return s
        
    def over_capacity(self):
        """ Returns True if node is over capacity """
        return len(self.items) > self.capacity
        
    def insert(self, tup):
        """
        Checks if tup can be inserted. If it can, insert leaf node into items
        and sort items. (easier than locating proper index to insert)
        
        O(n log(n)) instead of O(n) so this method is fine for small n
        """
        if tup in self.items:
            raise Exception(f"Can not insert tuple, {tup}, already exists")
        self.items.append(tup)
        self.items.sort()
        
    def split(self):
        """ splits a leaf node in place """
        # split items
        new_right = LeafNode(self.capacity)
        new_right.items = self.items[self.capacity//2 + 1:]
        
        self.items = self.items[:self.capacity//2 + 1]
        
        # change siblings
        new_right.rsibling = self.rsibling
        self.rsibling = new_right
        
        return new_right

In [4]:
from random import shuffle

In [5]:
intn_cap = 5
leaf_cap = 3

In [6]:
BPT = BPTree(intn_cap, leaf_cap)
numbers = list(range(30))
shuffle(numbers)
for num in numbers:
    BPT.insert(num)

In [7]:
BPT

Root: Internal Node
Capacity: 5
Items: [14, 22]
Children: [
    Internal Node
    Capacity: 5
    Items: [2, 4, 6, 8, 11]
    Children: [
        Leaf Node
        Capacity: 3
        Items: [0, 1]
    
        Leaf Node
        Capacity: 3
        Items: [2, 3]
    
        Leaf Node
        Capacity: 3
        Items: [4, 5]
    
        Leaf Node
        Capacity: 3
        Items: [6, 7]
    
        Leaf Node
        Capacity: 3
        Items: [8, 9, 10]
    
        Leaf Node
        Capacity: 3
        Items: [11, 12, 13]
    ]

    Internal Node
    Capacity: 5
    Items: [16, 19]
    Children: [
        Leaf Node
        Capacity: 3
        Items: [14, 15]
    
        Leaf Node
        Capacity: 3
        Items: [16, 17, 18]
    
        Leaf Node
        Capacity: 3
        Items: [19, 20, 21]
    ]

    Internal Node
    Capacity: 5
    Items: [24, 26, 28]
    Children: [
        Leaf Node
        Capacity: 3
        Items: [22, 23]
    
        Leaf Node
        Capacity: 3


In [8]:
BPT.depth()

2

In [9]:
in_ord_trav = list(BPT.in_order_traversal())
for i in range(len(in_ord_trav) - 1):
    assert(in_ord_trav[i] < in_ord_trav[i+1])